In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os

In [4]:
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

In [5]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	# otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
	else:
		cv2.normalize(hist, hist)
	# return the flattened histogram as the feature vector
	return hist.flatten()

In [6]:
def extract_sift_features(image, vector_size=32):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        kps, des = sift.detectAndCompute(gray, None)

        # If there are no keypoints or descriptors, return zero vector
        if des is None:
            return np.zeros(vector_size * 128)

        # Sort keypoints by response and keep the top N
        kps = sorted(kps, key=lambda x: -x.response)
        des = des[:vector_size]  # keep top-N descriptors

        # Flatten if not enough descriptors
        if des.shape[0] < vector_size:
            padding = np.zeros((vector_size - des.shape[0], 128))
            des = np.vstack((des, padding))

        return des.flatten()

    except cv2.error as e:
        print("SIFT error:", e)
        return np.zeros(vector_size * 128)


In [7]:
# Define the path to your dataset directly
dataset_path = "/content/drive/MyDrive/Colab Notebooks/aps360/Lab3_Gestures_Summer" # Replace with the actual path to your dataset
args = {
    "neighbors": 3,  # you can change k here
    "jobs": -1       # use all CPU cores
}

In [8]:
# grab the list of images that we'll be describing
print("[INFO] describing images...")
imagePaths = list(paths.list_images(dataset_path))
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
siftFeatures = []
combinedFeatures = []
labels = []

[INFO] describing images...


In [9]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    label = imagePath.split(os.path.sep)[-2].strip()

    # Extract features
    pixels = image_to_feature_vector(image)
    hist = extract_color_histogram(image)
    sift = extract_sift_features(image)
    combined = np.hstack([hist, sift])

    # Append to lists
    rawImages.append(pixels)
    features.append(hist)
    siftFeatures.append(sift)
    combinedFeatures.append(combined)
    labels.append(label)

    if i > 0 and i % 500 == 0:
        print(f"[INFO] processed {i}/{len(imagePaths)}")

[INFO] processed 500/2219
[INFO] processed 1000/2219
[INFO] processed 1500/2219
[INFO] processed 2000/2219


In [10]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
siftFeatures = np.array(siftFeatures)
combinedFeatures = np.array(combinedFeatures)
labels = np.array(labels)
print("[INFO] Data shapes:")
print(f"Raw Pixels: {rawImages.shape}")
print(f"Color Histograms: {features.shape}")
print(f"SIFT: {siftFeatures.shape}")
print(f"Combined: {combinedFeatures.shape}")
print(f"Labels: {labels.shape}")

[INFO] Data shapes:
Raw Pixels: (2219, 3072)
Color Histograms: (2219, 512)
SIFT: (2219, 4096)
Combined: (2219, 4608)
Labels: (2219,)


In [11]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(rawImages, labels, test_size=0.25, random_state=42)
(trainHist, testHist, trainHistLabels, testHistLabels) = train_test_split(features, labels, test_size=0.25, random_state=42)
(trainSIFT, testSIFT, trainSIFTLabels, testSIFTLabels) = train_test_split(siftFeatures, labels, test_size=0.25, random_state=42)
(trainCombo, testCombo, trainComboLabels, testComboLabels) = train_test_split(combinedFeatures, labels, test_size=0.25, random_state=42)

In [12]:
def evaluate_model(name, trainX, testX, trainY, testY):
    print(f"[INFO] evaluating {name} accuracy...")
    model = KNeighborsClassifier(n_neighbors=args["neighbors"], n_jobs=args["jobs"])
    model.fit(trainX, trainY)
    acc = model.score(testX, testY)
    print(f"[INFO] {name} accuracy: {acc * 100:.2f}%\n")

In [13]:
from sklearn.preprocessing import StandardScaler

# Normalize each feature set separately
scaler_raw = StandardScaler().fit(trainRI)
trainRI_norm = scaler_raw.transform(trainRI)
testRI_norm = scaler_raw.transform(testRI)

scaler_hist = StandardScaler().fit(trainHist)
trainHist_norm = scaler_hist.transform(trainHist)
testHist_norm = scaler_hist.transform(testHist)

scaler_sift = StandardScaler().fit(trainSIFT)
trainSIFT_norm = scaler_sift.transform(trainSIFT)
testSIFT_norm = scaler_sift.transform(testSIFT)

scaler_combo = StandardScaler().fit(trainCombo)
trainCombo_norm = scaler_combo.transform(trainCombo)
testCombo_norm = scaler_combo.transform(testCombo)

# Then use the normalized data in evaluate_model:
evaluate_model("Raw Pixel", trainRI_norm, testRI_norm, trainRL, testRL)
evaluate_model("Color Histogram", trainHist_norm, testHist_norm, trainHistLabels, testHistLabels)
evaluate_model("SIFT", trainSIFT_norm, testSIFT_norm, trainSIFTLabels, testSIFTLabels)
evaluate_model("Color Histogram + SIFT", trainCombo_norm, testCombo_norm, trainComboLabels, testComboLabels)

[INFO] evaluating Raw Pixel accuracy...
[INFO] Raw Pixel accuracy: 53.69%

[INFO] evaluating Color Histogram accuracy...
[INFO] Color Histogram accuracy: 32.97%

[INFO] evaluating SIFT accuracy...
[INFO] SIFT accuracy: 14.77%

[INFO] evaluating Color Histogram + SIFT accuracy...
[INFO] Color Histogram + SIFT accuracy: 17.84%

